In [25]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
train = pd.read_csv('final_train.csv')
test = pd.read_csv('final_test.csv')

In [28]:
#Dropping the unnecessary columns
train = train.drop(['oem','variant', 'model', 'Height', 'Wheel Base'], axis = 1)#height is incorrectly labelled or recorded
test = test.drop(['oem', 'variant', 'model', 'Height', 'Wheel Base'], axis = 1)

In [29]:
#Creating X_train and y_train
X_train = train.drop('listed_price', axis = 1)
y_train = train['listed_price']
X_test = test.drop('listed_price', axis = 1)
y_test = test['listed_price']
y_test

0        950000.0
1        500000.0
2        738000.0
3        660000.0
4        899999.0
          ...    
3770     295000.0
3771     735859.0
3772     245000.0
3773    1249999.0
3774     200000.0
Name: listed_price, Length: 3775, dtype: float64

In [33]:
catcols = [col for col in X_train if X_train[col].dtype == 'O']
catvals = [X_train[col].nunique() for col in catcols]
katie = pd.DataFrame({'catcols': catcols, 'Uniques' : catvals})
katie

,catcols,Uniques
0,transmission,2
1,fuel,5
2,Valve Configuration,5
3,Drive Type,4
4,owner_type,6
5,Fuel Supply System,10
6,body,11
7,Gear Box,11
8,Steering Type,2
9,Front Brake Type,5


In [34]:
#Adding avg_measure to the dataset
X_train['avg_measure'] = (X_train['Length'] + X_train['Width']) / 2
X_test['avg_measure'] = (X_test['Length'] + X_test['Width']) / 2

In [35]:
#Defining features
features = ['top_features',
 'comfort_features',
 'interior_features',
 'exterior_features',
 'safety_features']
X_test['Features'] = np.sum(X_test[features], axis = 1)
X_train['Features'] = np.sum(X_train[features], axis = 1)

In [38]:
#Valves featur
X_train['Valves'] = X_train['No of Cylinder'] * X_train['Valves per Cylinder']
X_test['Valves'] = X_test['No of Cylinder'] * X_test['Valves per Cylinder']


In [39]:
#Tread Features
X_train['Tread'] = (X_train['Front Tread'] + X_train['Rear Tread'])/ 2
X_test['Tread'] = (X_test['Front Tread'] + X_test['Rear Tread'])/ 2

In [40]:
to_drop = ['top_features', 'No of Cylinder', 'Valves per Cylinder',
       'comfort_features', 'interior_features', 'exterior_features',
       'safety_features','Length', 'Width', 'Front Tread', 'Rear Tread']

In [41]:
#Dropping the columns
X_train = lX_train.drop(to_drop, axis = 1)
X_test = lX_test.drop(to_drop, axis = 1)

In [43]:
lcat = [col for col in X_train.columns if X_train[col].dtype == 'O']

In [44]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
tra = pd.DataFrame(enc.fit_transform(X_train[lcat]))
tes = pd.DataFrame(enc.transform(X_test[lcat]))

/home/apsit/nayasaap/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [45]:
#Setting index
tra.index = X_train.index
tes.index = X_test.index

In [46]:
#Dropping the categorical columns
X_train = X_train.drop(lcat, axis = 1)
X_test = X_test.drop(lcat, axis = 1)

In [47]:
#Combining the columns back again
X_train = pd.concat([X_train, tra], axis = 1)
X_test = pd.concat([X_test, tes], axis = 1)

In [49]:
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [57]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
rf = RandomForestRegressor()

In [58]:
%%time
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
score = mean_absolute_error(y_test, preds)
score_percent = score * 100 / y_test.mean()
print("MAE: ", score)
print("MAE percent: ", score_percent)

ValueError: Found input variables with inconsistent numbers of samples: [3775, 1887]